In [ ]:
import psycopg2

NAME = 'projeto_mc536'
USER = 'postgres'
PASSWORD = 'sua_senha_aqui'
HOST = 'localhost'
PORT = '5432'

# Caminhos para os arquivos CSV utilizados
ENERGY_CSV = 'energias.csv'

In [8]:
def create_connection():
	"""Cria uma conexão com o banco de dados PostgreSQL."""
	try:
		conn = psycopg2.connect(
			dbname=NAME,
			user=USER,
			password=PASSWORD,
			host=HOST,
			port=PORT
		)
		return conn
	except Exception as e:
		print(f"Erro ao conectar ao banco de dados: {e}")
		return None

In [9]:
def close_cursor(cursor):
	"""Fecha o cursor."""
	if cursor:
		cursor.close()
		print("Cursor fechado.")

In [10]:
def end_connection(conn):
	"""Faz commit e encerra a conexão com o banco de dados."""
	if conn:
		conn.commit()
		conn.close()
		print("Conexão encerrada.")

In [11]:
def create_tables(conn):
	"""Cria todas as tabelas seguindo o modelo relacional."""
	cursor = conn.cursor()
	try:
		cursor.execute("""
			-- This script was generated by the ERD tool in pgAdmin 4.
			-- Please log an issue at https://github.com/pgadmin-org/pgadmin4/issues/new/choose if you find any bugs, including reproduction steps.
			BEGIN;


			CREATE TABLE IF NOT EXISTS public.pais
			(
				codigo character varying(5) NOT NULL,
				nome character varying(50) NOT NULL,
				continente character varying(20) NOT NULL,
				PRIMARY KEY (codigo)
			);

			CREATE TABLE IF NOT EXISTS public.grupo
			(
				id_grupo integer NOT NULL,
				sigla character varying(10) NOT NULL,
				PRIMARY KEY (id_grupo)
			);

			CREATE TABLE IF NOT EXISTS public.pais_grupo
			(
				pais_codigo character varying(5) NOT NULL,
				grupo_id_grupo integer NOT NULL
			);

			CREATE TABLE IF NOT EXISTS public.area
			(
				id integer NOT NULL,
				tipo character varying(10) NOT NULL,
				PRIMARY KEY (id)
			);

			CREATE TABLE IF NOT EXISTS public.distribuicao_territorial
			(
				pais_codigo character varying(5) NOT NULL,
				area_id integer NOT NULL,
				ano integer NOT NULL,
				area_km2 double precision NOT NULL
			);

			CREATE TABLE IF NOT EXISTS public.pib
			(
				ano integer NOT NULL,
				valor_usd double precision NOT NULL,
				codigo_pais character varying NOT NULL
			);

			CREATE TABLE IF NOT EXISTS public.demografia
			(
				area_id integer NOT NULL,
				pais_codigo character varying(5) NOT NULL,
				ano integer,
				num_habitantes integer
			);

			CREATE TABLE IF NOT EXISTS public.gas
			(
				id_gas integer NOT NULL,
				nome_gas character varying(100) NOT NULL,
				formula_gas character varying(10),
				PRIMARY KEY (id_gas)
			);

			CREATE TABLE IF NOT EXISTS public.emissao_gas
			(
				gas_id_gas integer NOT NULL,
				pais_codigo character varying(5) NOT NULL,
				ano integer NOT NULL,
				emissao_mtco2 double precision NOT NULL
			);

			CREATE TABLE IF NOT EXISTS public.energia
			(
				id_energia integer NOT NULL,
				tipo character varying(100) NOT NULL,
				PRIMARY KEY (id_energia)
			);

			CREATE TABLE IF NOT EXISTS public.interacao_energetica
			(
				id_interacao integer NOT NULL,
				nome character varying(100) NOT NULL,
				unidade character varying(10) NOT NULL,
				PRIMARY KEY (id_interacao)
			);

			CREATE TABLE IF NOT EXISTS public.relatorio_energetico
			(
				codigo_pais character varying(5) NOT NULL,
				id_interacao integer NOT NULL,
				id_energia integer NOT NULL,
				ano integer NOT NULL,
				valor double precision NOT NULL
			);

			ALTER TABLE IF EXISTS public.pais_grupo
				ADD FOREIGN KEY (pais_codigo)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.pais_grupo
				ADD FOREIGN KEY (grupo_id_grupo)
				REFERENCES public.grupo (id_grupo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.distribuicao_territorial
				ADD FOREIGN KEY (pais_codigo)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.distribuicao_territorial
				ADD FOREIGN KEY (area_id)
				REFERENCES public.area (id) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.pib
				ADD FOREIGN KEY (codigo_pais)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.demografia
				ADD FOREIGN KEY (area_id)
				REFERENCES public.area (id) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.demografia
				ADD FOREIGN KEY (pais_codigo)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.emissao_gas
				ADD FOREIGN KEY (gas_id_gas)
				REFERENCES public.gas (id_gas) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.emissao_gas
				ADD FOREIGN KEY (pais_codigo)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.relatorio_energetico
				ADD FOREIGN KEY (codigo_pais)
				REFERENCES public.pais (codigo) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.relatorio_energetico
				ADD FOREIGN KEY (id_interacao)
				REFERENCES public.interacao_energetica (id_interacao) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;


			ALTER TABLE IF EXISTS public.relatorio_energetico
				ADD FOREIGN KEY (id_energia)
				REFERENCES public.energia (id_energia) MATCH SIMPLE
				ON UPDATE NO ACTION
				ON DELETE NO ACTION
				NOT VALID;

			END;
		""")
		print("Tabelas criadas com sucesso.")
	except Exception as e:
		print(f"Erro ao criar tabelas: {e}")
	finally:
		close_cursor(cursor)

In [15]:
def check_existing_tables(conn):
	cursor = conn.cursor()
	try:
		cursor.execute("""
			SELECT tablename
			FROM pg_catalog.pg_tables
			WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
		""")
		tables = cursor.fetchall()
		if tables:
			print("Tabelas existentes:")
			for table in tables:
				print(table[0])
		else:
			print("Nenhuma tabela encontrada.")
	except Exception as e:
		print(f"Erro ao verificar tabelas existentes: {e}")
	finally:
		close_cursor(cursor)

In [17]:
conn = create_connection()
if conn:
		create_tables(conn)
		check_existing_tables(conn)
		end_connection(conn)

Tabelas criadas com sucesso.
Cursor fechado.
Tabelas existentes:
pais
pais_grupo
grupo
distribuicao_territorial
area
pib
demografia
gas
emissao_gas
relatorio_energetico
interacao_energetica
energia
Cursor fechado.
Conexão encerrada.
